<a href="https://colab.research.google.com/github/enrjorarr/ProyectoIA/blob/master/RedEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [9]:
#Conexión con google drive

from google.colab import drive
drive.mount('/content/drive')
#Rutas de paquetes de datos.
 
train_path = '/content/drive/My Drive/ProyectoIA/Datos/train'
test_path = '/content/drive/My Drive/ProyectoIA/Datos/test'
val_path = '/content/drive/My Drive/ProyectoIA/Datos/val'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Datos

epocas = 6
altura, longitud = 100, 100
batch_size = 32
pasos = 1000
pasos_validacion = 200
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3,3)
tamano_filtro2 = (2,2)
tamano_pool = (2,2)
clases = 9
lr = 0.005

In [0]:
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True
)

validacion_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [12]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    train_path,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 12002 images belonging to 9 classes.


In [13]:
imagen_validacion = validacion_datagen.flow_from_directory(
    val_path,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 3005 images belonging to 9 classes.


In [16]:
Adam = tf.keras.optimizers.Adam(
    learning_rate=lr,
    name='Adam'
)

cnn = Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape = (altura, longitud, 3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

#cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))

#cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())

cnn.add(Dense(64, activation='relu'))

cnn.add(Dropout(0.5))

cnn.add(Dense(64, activation='relu'))

cnn.add(Dropout(0.5))

cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss = 'categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

cnn.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                5120064   
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [17]:
cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

cnn.save('/content/drive/My Drive/ProyectoIA/Modelos/modelo.h5')
cnn.save_weights('/content/drive/My Drive/ProyectoIA/Modelos/pesos.h5')

Epoch 1/6
 376/1000 [==========>...................] - 6235s 17s/step - loss: 1.8853 - accuracy: 0.5119 - val_loss: 1.5371 - val_accuracy: 0.5245
